# Making a version of *Quantum Snake*
Q-Snake was a game created at the Qiskit Camp Europe hackathon in September 2019. In this notebook we create a new game, which take the basic idea of the original *Snake* and ports it to the PewPew

In [1]:
%matplotlib notebook

## Creating the head
In this game, the player controls a moving snake that moves around a 8x8 grid. The first thing we need to create is a moving dot that will represent the head of the snake. We will set up the controls. The snake cannot stop moving so our only actions are changing the direction of movement of the snake. For that we will use the keys  ▲, ▼, ◄ and ►. Rememeber that the snake cannot move backwards so in each step of the game the player has only two possible options.

In [ ]:
import pew
pew.init()
screen=pew.Pix()
# First we set variables of the game:

# The snake will be represented by a list of (x,y) coordinates. The last element of the list (snake[-1]) will
# represent the head of the snake. The 0th element of the list will be the last pixel of the tail. 

# Let's create the list by setting the initial position of the head at (3,3):
snake=[(3,3)]
# Now we create a variable to store the speed of the movement:
game_speed=5
# We also have to define the initial velocity of the snake. Let's set it to move in the x direction by 1 pixel.
dx, dy= 1, 0
# First we create and endless loop for the movement of the snake
while True:
    x,y=snake[-1] # We store the current position of the head in two variables x and y.
    screen.pixel(x, y, 3) # This command turns on the pixel (x,y) with the maximum brightnes (3) to print on
    # on the screen the current position of the head
    pew.show(screen) # Lights up the active pixels on the screen. We need to pew.show(screen) each frame of the game.
    pew.tick(1/game_speed) # Sets the refresh rate. When bigger "game_speed" the lower the refresh rate and the game
    # runs faster
# Now we have to check for user input by looking which keys are pressed. Depending on which keys are pressed the velocity of
# of the snake will change.
    keys=pew.keys() # Return a number telling which keys (or buttons) have been pressed since the
    # last check.  The number can then be filtered with the ``&`` operator and
    # the ``K_X``, ``K_DOWN``, ``K_LEFT``, ``K_RIGHT``, ``K_UP``, and ``K_O``
    # constants to see whether any of the keys was pressed.
    if keys & pew.K_UP and dy == 0:
            dx, dy = 0, -1
            # When the snake is moving in x and wee press UP the velocity of the snake changes to go up in the screen. 
            # Note that in the pew-pew screen the index of the top row is 0 and the index of the bottom row is 7.
            # This explains why we have dy=-1 when we press UP. The rest of the keys follow a similar logic.
    elif keys & pew.K_LEFT and dx == 0:
            dx, dy = -1, 0
    elif keys & pew.K_RIGHT and dx == 0:
            dx, dy = 1, 0
    elif keys & pew.K_DOWN and dy == 0:
            dx, dy = 0, 1
    # Now we calculate the next position of the snake's head:
    x=(x+dx)%8
    y=(y+dy)%8
    # We insert %8 to apply mod 8 to the coordinates, so when we pass through one of the walls we restart the counting and
    # appear in the other side of the screen.
    
    # We add to the list representing our snake a the next position of the head.
    snake.append((x,y))
    #We delete from the list the last position of the snake (in this case the head) and turn of that pixel
    (x,y)=snake.pop(0)
    screen.pixel(x,y,0)

## Putting some boundaries 

Now we have a controllable pixel that moves like the snake moves. But this is very boring to play! Let's add a "Game Over" mechanism: when we hit the walls we die and a Game Over screen appears. To do so we just need to add a few lines of code.

In [ ]:
import pew
pew.init()
screen=pew.Pix()
snake=[(3,3)]
game_speed=5
dx, dy= 1, 0
while True:
    x,y=snake[-1]
    screen.pixel(x, y, 3)
    pew.show(screen)
    pew.tick(1/game_speed)
    keys=pew.keys()
    if keys & pew.K_UP and dy == 0:
            dx, dy = 0, -1
    elif keys & pew.K_LEFT and dx == 0:
            dx, dy = -1, 0
    elif keys & pew.K_RIGHT and dx == 0:
            dx, dy = 1, 0
    elif keys & pew.K_DOWN and dy == 0:
            dx, dy = 0, 1
    #To implement the crashing mechanism we first have to get rid from the periodic screen, so we remove %8
    x=(x+dx)
    y=(y+dy)
    #Now add an if condition that breaks the while loop if any of the coordinates for the new head goes out from the grid.
    if x >7 or y > 7 or x < 0 or y < 0:
        # We create a loop for turning of all the pixels that forms the screen.
            for (i,j) in snake:
                screen.pixel(i,j,0)
            break
    snake.append((x,y))
    (x,y)=snake.pop(0)
    screen.pixel(x,y,0)
# Now that we are out the loop we show the "Game Over" screen:
text = pew.Pix.from_text("Game over!")
for dx in range(-8, text.width):
    screen.blit(text, -dx, 1)
    pew.show(screen)
    pew.tick(1 / 12) 

## Feeding the snake with quantum Apples

Ok, now we have a play ground, a controllable snake and a dying mechanism. We want our snake to grow so we need to feed it! To do so we are going to generate randomly apples on the grid. For such random generation we are going to use measurments on qubits using a simplified version of Qiskit that can run in a Pew-Pew: Aether. 
In Aether we have the limitation that we can use only two qubits, but that's no problem at all, we only need one!

To generate the apples we need 3 random bits for each coordinate (with 3 bits we have 2^3=8 different combinations, just the size of our grid). So in total we need 6 random bits. Let's see first how we can get a random bit using measurements on one qubit and then we will repeat the process 6 times.

### Using a qubit to generate a random bits

To generate a random bit we just need to:
* Initialize a qubit in the state |0>.
* Apply a gate H to the qubit to transforms its state to H|0>=1/sqrt(2)*(|0>+|1>)
* We measure the qubit in the basis (|0>,|1>) and we will get with probability 1/2 the result 1 and with probability 1/2 the result 0.

Let's see how to implement that process in Aether to get a random integer of 3 bits:

In [ ]:
from aether import QuantumCircuit, simulate 
def qrand(nbits):
    # We import the necessary elements form the aether library
    circ= QuantumCircuit(1,1) # We create a circuit with 1 qubit register and 1 classical bit register
    circ.h(0) # We apply the Hadamard gate to the qubit (note that by default the qubits are already intializated at 0)
    circ.measure(0, 0) # We make a measurement in the qubit and we store the result in the classical bit
    val = simulate(circ, shots=nbits, get='memory') # We simulate three times the circuit and we store the results in a list
    # We create a string with the binary number to use the function int() to obtain the integer
    b='' 
    for i in range(nbits):
        b+= str(val[i])
    # Trnsform the string into an int
    integ=int(b,2)
    # Return the integer
    return integ

### Generating the apples

Now that we have a quantum random number generator we can use it to generate the apples as follow: each time the snake eats an apple we call the function to simulate the circuit obtain the random coordinates for the apple. It might take several attempts because we might get invalid coordinates (for example if the apple lies in the body of the snake).

The implementation of the enlargement of the snake is very easy. We just need to insert the snake.pop(0) that removed the last part of the snake under the condition that the snake doesn't eat an apple. Otherwise that part won't be removed (but we will add the next position of the head) resulting in a snake of size +1.

Now that the snake can be bigger than a point it can hit with itself. We should also implement the death of the snake in that situation. 

In [ ]:
import pew
from aether import QuantumCircuit, simulate

# We define the function that we just created at the beginning of our code
def qrand(nbits):
    circ= QuantumCircuit(1,1)
    circ.h(0)
    circ.measure(0, 0)
    val = simulate(circ, shots=nbits, get='memory')
    b='' 
    for i in range(nbits):
        b+= str(val[i])
    integ=int(b,2)
    return integ

pew.init()
screen=pew.Pix()
snake=[(3,3)]
game_speed=5
dx, dy= 1, 0
# We must define the initial position of the apple:
apple_x, apple_y = 6, 6
screen.pixel(apple_x, apple_y, 3)

while True:
    x,y=snake[-1]
    screen.pixel(x, y, 3)
    pew.show(screen)
    pew.tick(1/game_speed)
    keys=pew.keys()
    if keys & pew.K_UP and dy == 0:
            dx, dy = 0, -1
    elif keys & pew.K_LEFT and dx == 0:
            dx, dy = -1, 0
    elif keys & pew.K_RIGHT and dx == 0:
            dx, dy = 1, 0
    elif keys & pew.K_DOWN and dy == 0:
            dx, dy = 0, 1
    x=(x+dx)
    y=(y+dy)
    # We need to add the condition that if the next position of the snake's head is in the snake's body the player dies
    if (x,y) in snake or x >7 or y > 7 or x < 0 or y < 0:
        # We create a loop for turning of all the pixels that forms the screen.
            for (i,j) in snake:
                screen.pixel(i,j,0)
            break
    snake.append((x,y))
    # In this loop we implement the generation of the apples each time the next position of the snake's head
    # lies into the current position of the apple
    if x == apple_x and y == apple_y:
        screen.pixel(apple_x, apple_y, 0)
        # We try the random generation of the apple until the apple appears out of the snake
        apple_x, apple_y = snake[0]
        while (apple_x, apple_y) in snake:
            apple_x=qrand(3)
            apple_y=qrand(3)
        screen.pixel(apple_x, apple_y, 2)
        # We also increase the velocity of the game each time the snake eats an apple
        gm_sp += 0.2
    # Otherwise we remove the tail of the snake
    else:
        x, y = snake.pop(0)
        screen.pixel(x, y, 0)
text = pew.Pix.from_text("Game over!")
for dx in range(-8, text.width):
    screen.blit(text, -dx, 1)
    pew.show(screen)
    pew.tick(1 / 12) 

## Some refinements
Now we have a working version of the game, but let's add some things:
* We may want to have a punctuation at the end of the game to see how many apples we ate. To do this we are going to add a counter that sums one each time we eat an apple.
* We also want the game to restart automatically each time we die. To achieve that we can put a loop while outside the whole code.

In [ ]:
import pew
from aether import QuantumCircuit, simulate
def qrand(nbits):
    circ= QuantumCircuit(1,1)
    circ.h(0)
    circ.measure(0, 0)
    val = simulate(circ, shots=nbits, get='memory')
    b='' 
    for i in range(nbits):
        b+= str(val[i])
    integ=int(b,2)
    return integ
pew.init()
screen=pew.Pix()
l=True # We initialize a boolean that will help us to reset the initial conditions of the game
while True:
    #Now we put the condition of reseting the initial variables only if l=True
    if l: 
        game_speed=5
        points=0 # We initialize a variable to count the points
        snake = [(3,3)]
        dx, dy = 1, 0
        apple_x, apple_y = 6, 6
        screen.pixel(apple_x, apple_y, 3)
        l=False
    while True:
        x,y=snake[-1]
        screen.pixel(x, y, 3)
        pew.show(screen)
        pew.tick(1/game_speed)
        keys=pew.keys()
        if keys & pew.K_UP and dy == 0:
                dx, dy = 0, -1
        elif keys & pew.K_LEFT and dx == 0:
                dx, dy = -1, 0
        elif keys & pew.K_RIGHT and dx == 0:
                dx, dy = 1, 0
        elif keys & pew.K_DOWN and dy == 0:
                dx, dy = 0, 1
        x=(x+dx)
        y=(y+dy)
        if (x,y) in snake or x >7 or y > 7 or x < 0 or y < 0:
                for (i,j) in snake:
                    screen.pixel(i,j,0)
                break
        snake.append((x,y))
        if x == apple_x and y == apple_y:
            screen.pixel(apple_x, apple_y, 0)
            apple_x, apple_y = snake[0]
            while (apple_x, apple_y) in snake:
                apple_x=qrand(3)
                apple_y=qrand(3)
            screen.pixel(apple_x, apple_y, 2)
            game_speed += 0.2
            # We add one to the points counter each time we eat an apple
            points=points+1
        else:
            x, y = snake.pop(0)
            screen.pixel(x, y, 0)
    l=True # We set l to True to restart to the initial variables in the next iteration
    #Now let's display the points in the screen:
    text = pew.Pix.from_text('Points: ' + str(points))
    for dx in range(-8, text.width):
        screen.blit(text, -dx, 1)
        pew.show(screen)
        pew.tick(1 / 12)
    text = pew.Pix.from_text("Game over!")
    for dx in range(-8, text.width):
        screen.blit(text, -dx, 1)
        pew.show(screen)
        pew.tick(1 / 12) 

## Exercise 1
Add a barrier in the middle of the grid so that each time the snake hits the barrier it dies or passes through with some probability. Use aether to calculate the outcome of the event.

## Exercise 2
Unfortunately we don't have a real quantum computer inside the pew-pew. This means that the apples are not generated truly randomly but using the pseudo-random features of Python. If you play a lot you will notice that if you do the same movements the apples appear exactly in the same spots. To avoid this implement a method to change the random seed based on the input of the player.